# Lecture 3: Password Attack with Difference Analysis - Introduction

In this example we want to improve the basic passwordcheck to be resistant against the attack from the last tutorial.

## Improving the code

Let's first recap the password checking loop from the basic passwordcheck:
```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != input[i])
    {
        password_wrong = 1;
        break;
    }
}
```

The timing attack discussed in the last example worked because the loop's runtime varies with the number of correct characters. Once the first wrong character occurs the loop breaks.
This is, what we want to change:

```c
for(uint8_t i = 0; i < sizeof(stored_password); i++)
{
    if (stored_password[i] != input[i])
    {
        password_wrong = 1;
    }
}
```

This is an excerpt from `3_password_fixed.c`. It is clear that the loop does not break after the first wrong character and always all characters of the password are checked.

## Develop a new idea

In [1]:
import sys
sys.path.insert(0, '..')
import securec
from securec import util
scope, target = util.init()

In [2]:
securec.util.compile_and_flash('./3_password_fixed.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2045 bytes
✓


In [3]:
scope.default_setup()
scope.adc.samples = 500

def capture(attempt, count=1):
    if isinstance(attempt, str):
        attempt = attempt.encode('iso-8859-1')
    elif isinstance(attempt, int):
        attempt = bytes([attempt])

    scope.arm()
    target.simpleserial_write(0x01, attempt + b' ' * (8 - len(attempt)))
    result = target.simpleserial_read(0x01, 1)
    return util.capture(), not bool(result[0])

In [4]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10, Category20_20

output_notebook()

Loading BokehJS ...

In [5]:
traces = [(attempt, capture(attempt)[0]) for attempt in (' ', 'a', 'i')]

ERROR:ChipWhisperer Target:Device did not ack


In [6]:
p = figure(width=900, height=200)
p.add_tools(CrosshairTool())
for idx, ((attempt1, trace1), (attempt2, trace2)) in enumerate([
    (traces[0], traces[1]),
    (traces[0], traces[2]),
]):
    p.line(
        range(len(trace1)), 
        abs(trace1 - trace2), 
        line_color=Category10_10[idx],
    )
show(p)

## MAD password attack

Given two traces $t_1, t_2$ their MAD value is defined as:
$$\text{MAD}(t_1, t_2) := \text{max}\big( \big| t_1[i] - t_2[i] \big| \big),$$
where $t[i]$ is the value of trace $t$ at point $i$.

In [7]:
def mad(trace1, trace2):
    return max(abs(trace1 - trace2))

In [8]:
def attack_password_mad(
    charlist='abcdefghijklmnopqrstuvwxyz',
    basechar='-',
):
    result = False
    password = ''

    while not result and len(password) < 8:
        basetrace, _ = capture(password + basechar)
        traces = [capture(password + c) for c in charlist]
        result = any((result for _, result in traces))
        mads = [mad(trace, basetrace) for trace, _ in traces]
        c = max(charlist, key=lambda c: mads[charlist.index(c)])
        password += c
        print(f'success: "{c}" => password = {password}')
    return password, result

In [9]:
attack_password_mad()

success: "i" => password = i
success: "n" => password = in
success: "f" => password = inf
success: "i" => password = infi
success: "n" => password = infin
success: "e" => password = infine
success: "o" => password = infineo
success: "n" => password = infineon


('infineon', True)

In [10]:
util.exit()